# Gerekli Kütüphaneleri Yükleme

Öncelikle veri işleme, ölçeklendirme, normalizasyon, özellik seçimi ve modelleme işlemleri için gerekli kütüphaneleri yüklüyoruz. Bu kütüphaneler, sayısal işlemlerden veri hazırlama ve özellik seçimine kadar pek çok işlemi yapmamızı sağlar.

In [140]:
import numpy as np
from pandas import read_csv
from numpy import set_printoptions
from sklearn.preprocessing import MinMaxScaler, Normalizer, Binarizer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from sklearn.decomposition import PCA
from sklearn.ensemble import ExtraTreesClassifier

# Veri Dosya Yolu ve Değişken İsimlerini Tanımlama

Veri dosyasının yolunu belirleyip, veri setindeki sütunları anlamlı isimlerle tanımlıyoruz. Bu sayede veri yükleme ve işleme sürecinde sütunları kolayca ayırt edebiliriz.

In [141]:
dosya = '/home/karsterr/Documents/CSIT/AI Programming/Datasets/pima_indians_diabetes.csv'
degiskenler = ['geb', 'glik', 'pres', 'deri', 'ins', 'bmi', 'soy', 'yas', 'sinif']

# Veri Kümesini Yükleme

*read_csv* fonksiyonu ile veri setini yüklüyoruz. Veri setinin ilk birkaç satırını ekrana yazarak dosyanın doğru şekilde okunduğunu kontrol ediyoruz.

In [142]:
veri = read_csv(dosya, names=degiskenler, header=0)
print(veri.head())

   geb  glik  pres  deri  ins   bmi    soy  yas  sinif
0    6   148    72    35    0  33.6  0.627   50      1
1    1    85    66    29    0  26.6  0.351   31      0
2    8   183    64     0    0  23.3  0.672   32      1
3    1    89    66    23   94  28.1  0.167   21      0
4    0   137    40    35  168  43.1  2.288   33      1


# Pandas'dan Numpy Dizisine Çevirme

Pandas DataFrame'ini Numpy dizisine çeviriyoruz. Bu işlem, veriyi Numpy tabanlı kütüphanelerle daha kolay işlememizi sağlar.

In [143]:
dizi = veri.values
print(dizi)

[[  6.    148.     72.    ...   0.627  50.      1.   ]
 [  1.     85.     66.    ...   0.351  31.      0.   ]
 [  8.    183.     64.    ...   0.672  32.      1.   ]
 ...
 [  5.    121.     72.    ...   0.245  30.      0.   ]
 [  1.    126.     60.    ...   0.349  47.      1.   ]
 [  1.     93.     70.    ...   0.315  23.      0.   ]]


# Girdi ve Çıktı Değişkenlerini Ayırma

Bağımsız değişkenleri *x* ve bağımlı değişkeni *y* olarak ayırıyoruz. Bu, veriyi modellemeye hazırlarken önemli bir adımdır.

In [144]:
x = dizi[:, 0:8]  # İlk 8 sütun özellikler
y = dizi[:, 8]    # Son sütun hedef değişken

# Min-Max Ölçekleme

Min-Max ölçekleyici kullanarak veriyi 0-1 aralığına dönüştürüyoruz. Bu işlem, özelliklerin aynı ölçek aralığında olmasını sağlayarak model performansını artırır.

In [145]:
olcekleyici = MinMaxScaler(feature_range=(0,1))
olceklendirilmis_x = olcekleyici.fit_transform(x)

# Ölçeklenmiş veriyi görüntüleme
set_printoptions(precision=3)
print(olceklendirilmis_x[5:10, :])

[[0.294 0.583 0.607 0.    0.    0.382 0.053 0.15 ]
 [0.176 0.392 0.41  0.323 0.104 0.462 0.073 0.083]
 [0.588 0.578 0.    0.    0.    0.526 0.024 0.133]
 [0.118 0.99  0.574 0.455 0.642 0.455 0.034 0.533]
 [0.471 0.628 0.787 0.    0.    0.    0.066 0.55 ]]


# Normalizasyon

Verinin her satırını birim uzunluğa getiriyoruz. Normalizasyon, özelliklerin farklı büyüklüklerden etkilenmeden işlenmesini sağlar.

In [147]:
normalizer = Normalizer().fit(x)
normalize_edilmis_x = normalizer.transform(x)

set_printoptions(precision=3, suppress=True)  # Bilimsel gösterimi kapatıyoruz
print(normalize_edilmis_x[5:10, :])

[[0.035 0.81  0.517 0.    0.    0.179 0.001 0.209]
 [0.022 0.566 0.363 0.232 0.638 0.225 0.002 0.189]
 [0.081 0.926 0.    0.    0.    0.284 0.001 0.234]
 [0.003 0.336 0.119 0.077 0.925 0.052 0.    0.09 ]
 [0.048 0.749 0.576 0.    0.    0.    0.001 0.324]]


# Veriyi İkili (Binarize) Hale Getirme

Eşik değerine göre veriyi ikili hale getiriyoruz. 0 eşik değeri altında kalan veriler 0, üstünde kalan veriler ise 1 olarak atanır.

In [148]:
ikili_donusum = Binarizer(threshold=0.0).fit(x)
ikilesmis_x = ikili_donusum.transform(x)

set_printoptions(precision=3, suppress=True)
print(ikilesmis_x[5:10, :])

[[1. 1. 1. 0. 0. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 0. 0. 0. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 0. 0. 0. 1. 1.]]


# Özellik Seçimi - Tek Değişkenli Seçim

Chi-Square testi ile hedef değişkenle en yüksek ilişkili olan en iyi 4 özelliği seçiyoruz. Bu işlem, gereksiz özellikleri eleyerek modelin daha iyi çalışmasını sağlar.

In [149]:
secici = SelectKBest(score_func=chi2, k=4)
x_yeni = secici.fit_transform(x, y)

set_printoptions(precision=3, suppress=True)
print(f"Seçilen Özellik İndeksleri : {secici.get_support(indices=True)}")

Seçilen Özellik İndeksleri : [1 4 5 7]


# Geriye Yönelik Özellik Eleme (Recursive Feature Elimination - RFE)

Lojistik regresyon modelini kullanarak özellikleri adım adım azaltıyoruz. En iyi özellik kombinasyonlarını bulmak için her adımda en düşük önem sırasındaki özellikleri kaldırıyoruz.

In [151]:
model = LogisticRegression(max_iter=1000)
rfe = RFE(model, step=4)

x_yeni = rfe.fit_transform(x, y)

print(f"Seçilen Özellik İndeksleri : {rfe.get_support(indices=True)}")
print(f"Yeni Özellikler: {x_yeni[5:10, :]}")

Seçilen Özellik İndeksleri : [0 1 5 6]
Yeni Özellikler: [[  5.    116.     25.6     0.201]
 [  3.     78.     31.      0.248]
 [ 10.    115.     35.3     0.134]
 [  2.    197.     30.5     0.158]
 [  8.    125.      0.      0.232]]


# Ana Bileşen Analizi (PCA) ile Özellik Seçimi

PCA ile verinin boyutunu azaltarak en yüksek varyansı açıklayan bileşenleri oluşturuyoruz. Bu sayede model performansı artırılırken veri karmaşıklığı azalır.

In [152]:
pca = PCA(n_components=4)
x_yeni = pca.fit_transform(x)

print(f"Yeni Özellikler : {x_yeni[5:10, :]}")
print(f"Açıklanan Varyans Oranı : {pca.explained_variance_ratio_}")


Yeni Özellikler : [[-80.978   3.947  -0.641 -15.118]
 [  4.351 -46.934  -9.183  11.509]
 [-82.137  -7.098 -67.489   6.131]
 [468.866  31.528 -10.057  -5.99 ]
 [-80.194  18.048  18.05  -32.127]]
Açıklanan Varyans Oranı : [0.889 0.062 0.026 0.013]


# Özellik Önem Dereceleri

Ekstra Ağaçlar sınıflandırıcısını kullanarak her bir özelliğin hedef değişkene katkısını hesaplıyoruz. Bu özelliklerin önem sıralamasına göre en yüksek dereceli olanları belirleyebiliriz.

In [153]:
model = ExtraTreesClassifier()
model.fit(x, y)

importances = model.feature_importances_
indices = np.argsort(importances)[::-1]

selected_indices = indices[:4]

set_printoptions(precision=3, suppress=True)
print(f"Yeni Özellikler : {x[5:10, selected_indices]}")
print(f"Açıklanan Varyans Oranı : {model.feature_importances_}")


Yeni Özellikler : [[116.     30.     25.6     0.201]
 [ 78.     26.     31.      0.248]
 [115.     29.     35.3     0.134]
 [197.     53.     30.5     0.158]
 [125.     54.      0.      0.232]]
Açıklanan Varyans Oranı : [0.108 0.231 0.097 0.082 0.074 0.144 0.119 0.144]
